# Package Tutorial

This is a friendly tutorial to cover a basic image classification task. We will leverage the capabilities of `dysweep` to run different experiments on all the different configurations of the model design, as well as the dataset.

First run the following piece of code to set the root directory of the project as the parent directory:

In [1]:
# set the parent directory as the working directory
import sys
sys.path.append("..")
# remove the current directory from the path
sys.path = sys.path[1:]

## Getting started

To get started, we should first imagine the entire pipeline of our experiment. In this simple case, we have to first download the datasets for classification (in this case CIFAR10), perform any preprocessing and transforms needed to create a dataloader. Furthermore, we will also pick a specific model designed for the task at hand and train it using an optimizer. 

Each of the different parts of the experiments have different hyperparameters and configurations. It is good to come up with a plan of how we want to structure our configurations beforehand. A simple sketch of the run configurations as well as the simple training loop itself is given below that logs all the results in `wandb`.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from tqdm import tqdm
import wandb

wandb.init(project="image_classification")

# Configurations and hyper-parameters

# Training configurations
EPOCH_COUNT = 10
LR = 0.001
# Data configurations
NUM_WORKERS = 2
BATCH_SIZE = 64
# Model configurations
NUM_CLASSES = 10
PRETRAINED_OR_NOT = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Define transformations for the train set
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Define transformations for the test set
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load datasets
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Create dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Load the ResNet model
model = resnet50(pretrained=PRETRAINED_OR_NOT, num_classes=NUM_CLASSES).to(device) 

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# Training function
def train(epoch_count=EPOCH_COUNT):
    model.train()
    for epoch in range(epoch_count):  # 10 epochs
        wandb.log({"epoch": epoch})
        for i, data in tqdm(enumerate(train_loader, 0)):
            inputs, labels = data[0].to(device), data[1].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            wandb.log({"loss": loss.item()})
            
    # compute the test accuracy
    correct = 0
    for i, data in tqdm(enumerate(test_loader, 0)):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        outputs = model(inputs)
        # get the maximum logit
        _, predicted = torch.max(outputs.data, 1)
        # check if the prediction is correct
        correct += (predicted == labels).sum().item()
    
    wandb.log({"accuracy": correct / len(test_set)})
 
try:   
    train()
finally:
    wandb.finish()
    

## Generic and Configurable using Dypy

To illustrate how our configurations work, we will make the above code configurable down to some of the very basic components. Usually, there is no need for such detailed configuration setup; however, we do this to showcase some features of out pipeline:

1. Outline some of the functionalities of the [DyPy](https://github.com/vahidzee/dypy) library.
2. Illustrate that the configurations can be complex and hierarchical, but generic in a sense that you can manipulate everything down to the very last detail using a configuration file.

Furthermore, we will define a YAML configuration file that will allow us to define the entire experiment, and that these experiments are tunable down to the very last details.

```YAML
data:
    dataset_class: torchvision.datasets.CIFAR10
    batch_size: 64
    num_workers: 2
    train_transforms:
    -   class_path: torchvision.transforms.RandomHorizontalFlip
    -   class_path: torchvision.transforms.RandomCrop
        init_args:
            size: 32
            padding: 4
    -   class_path: torchvision.transforms.ToTensor
    -   class_path: torchvision.transforms.Normalize
        init_args:
            mean: [0.4914, 0.4822, 0.4465]
            std: [0.2023, 0.1994, 0.2010]
    test_transforms:
    -   class_path: torchvision.transforms.ToTensor
    -   class_path: torchvision.transforms.Normalize
        init_args:
            mean: [0.4914, 0.4822, 0.4465]
            std: [0.2023, 0.1994, 0.2010]
model:
    class_path: torchvision.models.resnet50
    init_args:
        pretrained: False
        num_classes: 10
trainer:
    epoch_count: 10
    optimizer:
        class_path: torch.optim.SGD
        init_args:
            lr: 0.001
```

Now if we assume `config` is the above YAML configuration, we can re-write the code as follows:

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
from tqdm import tqdm
import dypy as dy
import yaml

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load conf.yaml into the dictionary config
with open("conf.yaml", 'r') as stream:
    cfg = yaml.safe_load(stream)

# Define transformations for the train set
train_transform = transforms.Compose([
    dy.eval(x['class_path'])(**(x['init_args'] if 'init_args' in x else {})) for 
        x in cfg['data']['train_transforms']
])

# Define transformations for the test set
test_transform = transforms.Compose([
    dy.eval(x['class_path'])(**(x['init_args'] if 'init_args' in x else {})) for 
        x in cfg['data']['test_transforms']
])

# Load datasets
train_set = dy.eval(cfg['data']['dataset_class'])(root='./data', train=True, download=True, transform=train_transform)
test_set = dy.eval(cfg['data']['dataset_class'])(root='./data', train=False, download=True, transform=test_transform)

# Create dataloaders
train_loader = torch.utils.data.DataLoader(
    train_set, 
    batch_size=cfg['data']['batch_size'], 
    shuffle=True, 
    num_workers=cfg['data']['num_workers'],
)
test_loader = torch.utils.data.DataLoader(
    test_set, 
    batch_size=cfg['data']['batch_size'], 
    shuffle=False, 
    num_workers=cfg['data']['num_workers'],
)

# Load the ResNet model
model = dy.eval(cfg['model']['class_path'])(**cfg['model']['init_args']).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = dy.eval(
        cfg['trainer']['optimizer']['class_path']
    )(model.parameters(), **cfg['trainer']['optimizer']['init_args'])
    
try:
    train(epoch_count=cfg['trainer']['epoch_count'])
finally:
    wandb.finish()

Now one can easily manipulate the YAML file to their needs to have all the different experiments run. Since the configuration is generic, a lot of different setups can be run. For example, you can define your custom set of transforms, or even define your own model and optimizer.

This is partially due to the dynamic nature of the [DyPy](https://github.com/vahidzee/dypy) library as well. Although not necessarily best practice, one can also define code snippets in the YAML file. This is useful for quick prototyping and testing all the possible configurations without touching the actual source code.

Furthermore, we will use this configuration as a template to change it automatically using the weights and biases sweep API.

## From Flat Sweeps to Hierarchical Sweeps

Here's where we will start using the weights and biases library and later extend it with our library. First off, say we want to sweep over the following different hyper-parameters:
1. Different learning rate values: `[0.001, 0.01]`.
2. Different optimizers: `[SGD, Adam]`.
3. Different epoch counts: `[10, 20]`.
4. Different batch sizes: `[32, 64]`.
5. For the Adam optimizer, we're going to try out different weight_decay values: `[0.1, 0.01]`.
Note that the standard sweep configuration dictionary is very limited, and should be defined in a flat way. Therefore, we will define the sweep configuration as follows:


In [ ]:
import wandb

sweep_config = {
    'name': 'my-sweep',
    'method': 'grid',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'   
    },
    'parameters': {
        'optimizer_type': {
            'values': ['torch.optim.SGD', 'torch.optim.AdamW']
        },
        'optimizer_weight_decay': {
            'values': [0.1, 0.01]  
        },
        'optimizer_lr': {
            'values': [0.001, 0.01],
        },
        'epoch_count': {
            'values': [10, 20],
        },
        'batch_size': {
            'values': [32, 64],
        },
    } 
}

sweep_id = wandb.sweep(sweep_config)

def train():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Update the cfg dictionary with sweep parameters
        cfg = run.config
        
        # change the sweep configuration into the elaborate, yet generic configuration
        with open("conf.yaml", 'r') as stream:
            base_cfg = yaml.safe_load(stream)
        base_cfg['trainer']['optimizer']['class_path'] = cfg['optimizer_type']
        base_cfg['trainer']['optimizer']['init_args']['lr'] = cfg['optimizer_lr']
        if cfg['optimizer_type'] == 'torch.optim.Adam':
            base_cfg['trainer']['optimizer']['weight_decay'] = cfg['optimizer_weight_decay']
        base_cfg['trainer']['epoch_count'] = cfg['epoch_count']
        base_cfg['data']['batch_size'] = cfg['batch_size']
        
        #... (insert your training code here) ...
        raise NotImplementedError("Implement your trainer using wandb logs from the code snippet above!")

# Run the sweep
wandb.agent(sweep_id, function=train, count=1)


Since the sweep configuration is flat, there should exist an extra few lines to convert the flat configuration into the generic one that was intended to work on all different running procedures. One of the many features of our library is that it eliminates that and allows W&B sweeps to also work with hierarchies.

We recommend working with the `dysweep_run_resume` function that takes in all the sweep configurations as well as a function that has the following signature: `func(config, checkpoint_dir)`. For brevity, we will not explain `checkpoint_dir` here and only focus our attention on `config`. Later on, we will return to that.

In this example, the `dysweep_run_resume` function will take in a base configuration as well as a sweep configuration as follows. The hierarchies also apply to the values that are being sweeped upon. Therefore, for instance, here we can see that an entire `optimizer` dictionary is being sweeped upon, in cotrast to only sweeping over primitve values.

**Note**: `dysweep` uses a decoy run under the project specified that contains specific meta-data to communicate between the vanilla W&B API and our new one. Therefore, the name of the project should be specified when called; otherwise, the decoy run will be created under a different project.

In [ ]:
from dysweep import dysweep_run_resume
from pprint import pprint
import yaml

with open("conf.yaml", 'r') as stream:
    base_cfg = yaml.safe_load(stream)

# Define the sweep configuration
sweep_config = {
    'name': 'my-sweep',
    'method': 'grid',
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'data': {
            'batch_size': {
                'sweep': True,
                'values': [32, 64, 128]
            },
        },
        'trainer': {
            'epoch_count': {
                'sweep': True,
                'values': [10, 20],
            },
            'optimizer': {
                'sweep': True,
                'sweep_alias': [
                    'adam-lr-0.001-wd-0.1',
                    'adam-lr-0.001-wd-0.01',
                    'adam-lr-0.01-wd-0.1',
                    'adam-lr-0.01-wd-0.01',
                    'sgd-lr-0.001',
                    'sgd-lr-0.01',
                ],
                'values': [
                    {
                        'class_path': 'torch.optim.Adam',
                        'init_args': {
                            'lr': 0.001,
                            'weight_decay': 0.1,
                        },  
                    },
                    {
                        'class_path': 'torch.optim.Adam',
                        'init_args': {
                            'lr': 0.001,
                            'weight_decay': 0.01,
                        },  
                    },
                    {
                        'class_path': 'torch.optim.Adam',
                        'init_args': {
                            'lr': 0.01,
                            'weight_decay': 0.1,
                        },  
                    },
                    {
                        'class_path': 'torch.optim.Adam',
                        'init_args': {
                            'lr': 0.01,
                            'weight_decay': 0.01,
                        },  
                    },
                    {
                        'class_path': 'torch.optim.SGD',
                        'init_args': {
                            'lr': 0.001,
                        },  
                    },
                    {
                        'class_path': 'torch.optim.SGD',
                        'init_args': {
                            'lr': 0.01,
                        },  
                    },
                ]
            }
        }
    }
}

sweep_id = dysweep_run_resume(
    project="tutorial",
    base_config=base_cfg,
    # A hierarchical sweep configuration that is not possible with the normal sweep library
    sweep_configuration= sweep_config,
)

The `dysweep_run_resume` function is multi-purpose! This means that after instantiating the sweep, you can use the same function with different arguments to run a specific configuration of that sweep. Note that you can run the following line of code **on any other machine** -- this is part of the beauty of sweep in general, that allows running configurations on different machines and even clusters for parallel computing. It should just simply have access to the same workspace and project to download all the meta-data required.

In [ ]:
from dysweep import dysweep_run_resume
from pprint import pprint

# sweep_id = # Enter the sweep id obtained

def train(config, checkpoint_dir):
    ### YOUR TRAINING CODE HERE (glossing over checkpoint_dir) ###
    print("Training on the following configuration:")
    pprint(config)
    raise NotImplementedError("Implement your trainer using wandb logs from the code snippet above!")


ret = dysweep_run_resume(
    project="tutorial",
    function=train,
    count=1,
    sweep_id=sweep_id,
)

### Using Upserts

Now if you take a look at the sweep configuration that was specified earlier, you will notice that you have to repeat a lot of configurations in the `optimizer` portion. In specific, for any configuration of `optimizer_type` and `weight_decay`, you would have to repeat the learning rates as well. This is not ideal, and we can use the `upsert` functionality to avoid this. The `upsert` functionality allows us to specify a list of configurations that will be merged with the base configuration. 

What we will do in this particular case is that we will first specify the `optimizer_type` and `weight_decay` (3 different ways). In turn, we will update the two different values for `lr` on top of that (2 different ways). Allowing us to come up with the same set of 6 different configurations, but in a much more concise way.

To do so, simply change the `sweep_config` to the following:

```python
{
    'name': 'my-sweep',
    'method': 'grid',
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'data': {
            'batch_size': {
                'sweep': True,
                'values': [32, 64, 128]
            },
        },
        'trainer': {
            'epoch_count': {
                'sweep': True,
                'values': [10, 20],
            },
            'optimizer': {
                'sweep': True,
                'sweep_alias': [
                    'adam-wd-0.1',
                    'adam-wd-0.01',
                    'sgd',
                ],
                'values': [
                    {
                        'class_path': 'torch.optim.AdamW',
                        'init_args': {
                            'weight_decay': 0.1,
                        },  
                    },
                    {
                        'class_path': 'torch.optim.AdamW',
                        'init_args': {
                            'weight_decay': 0.01,
                        },  
                    },
                    {
                        'class_path': 'torch.optim.SGD',
                    },
                ]
            },
            'dy__upsert': [
                {
                    'sweep': True,
                    'sweep_identifier': 'lr',
                    'sweep_alias': [
                        'lr-0.001',
                        'lr-0.01',
                    ],
                    'values': [
                        {
                            'optimizer': {
                                'init_args': {'lr': 0.001},
                            },
                        },
                        {
                            'optimizer': {
                                'init_args': {'lr': 0.01},
                            },
                        },
                    ]
                }
            ]
        }
    }
}
```


The entire philosophy of dysweep revolvs around upserting configurations and we have a lot of advanced dynamic functionalities to do so. You can read more about all the ways we can alter the sweep configuration in the [documentation](https://dysweep.readthedocs.io/en/latest/).

## Checkpointing and Re-running

While doing large-scale computing, the sweep server passes on sweep configurations to the goal machine for running. If by any chance, the goal machine fails, the sweep server will not be able to re-run the same configuration again. To do so, there should be a systematic repository that saves all the run identifiers that have finished, are being run, or have failed. 

With dysweep, all of this is handled seamlessly for you. In fact, for every configuration, a specific checkpoint directory is specified that you can use to store model checkpoints. In turn, whenever you try to re-run, you can load the model from the checkpoint directory and continue training from there. This is the reason why the `dysweep_run_resume` function contains a `resume` keyword, and that the function passed to it can take in an extra argument `checkpoint_dir`.

### Re-running

Whenever dysweep instantiates a new sweep, all the logs will be saved in either `./dysweep_logs` or in a specific root directory you specify. In this directory, a set of files will be created per each of the sweeps you create. For example, if your sweep has id equal to `s6kye2ah`, then a `checkpoints-s6kye2ah` directory will appear that contains meta-data and checkpoint of all the runs associated with that sweep.

Under this directory you will see two things:
1. A set of configurations that were run successfully. These `json` files contain the entire configuration that `func` was called with and are stored in `{run_id}-config.json` format.
2. A set of configurations that are being run currently and either have not finished, or have failed. These runs contain both configurations, and checkpoints. These information are stored in a subdirectory of the format `{i}-{run_id}`. The `i` is simply an index used internally for ordering the runs that are being run.
Using this subdirectory, you can access specific run ids that you wish to re-run. 

To do so, simply pass the `rerun_id` argument in the `dysweep_run_resume` function. For example, if you wish to re-run the run with `id=xipqslhf`, you can do so as follows:

```python
dysweep_run_resume(
    func=func,
    resume=True,
    count=10, # resume 10 runs
    project='tutorial',
)
```

### Resuming 

To resume a specific run, you can set the `resume` argument to `True`. Note that what this does is that whenever the function `func` is being called, the `checkpoint_dir` will be a directory that contains all of your previous checkpoints. From that point on, you can load the model from the checkpoint directory and continue training from there. 

### Multi-resume

Say you have run a sweep for a long time and you come back to your system only to realize 10 of the runs have failed due to the machine cluster preempting the tasks. You would want to resume all of the 10 runs **using a single process** on a machine that you know will not preempt. In that case, you can call the `dysweep_run_resume` function with the `resume` option set to `True`. However, instead of defining a single `rerun_id`, you will set the `count` argument to the number of runs you wish to resume. 

```python
dysweep_run_resume(
    func=func,
    rerun_id='xipqslhf',
    project='tutorial',
)
```

Using all of the information from this section you can play around with the training task to check all of these capabilities.

## Beyond Hyperparameter Search

Our aim with this library is to allow for a more generic and configurable way of running experiments. Therefore, we intend on using the library for **any** large scale computing involving different configurations. 

For example, let's assume you want to check your model performance across a variety of datasets. Instead of running different sweeps for each dataset, we can define a standard and generic configuration scheme for all possible datasets, and then plug them into our sweep configuration. This way, we can run our model on multiple datasets using a single sweep. 